In [1]:
import pandas as pd   
import re
import numpy as np
import matplotlib.pyplot as plt
import os   
import scipy
from nltk.corpus import stopwords
from gensim.models import KeyedVectors

from nltk import pos_tag, word_tokenize
import nltk
nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hezi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Hezi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
df = pd.read_csv("../../data/Paraphrase.csv")

In [3]:
df['bleu_allwords'] = 0
df['bleu_withoutstop'] = 0

In [4]:
df

,text_1,text_2,label,bleu_allwords,bleu_withoutstop
0,take measures in order to,take additional measures to,4.000,0,0
1,was seeking to,will strive to,3.000,0,0
2,canada aims to,canada works to,2.333,0,0
3,refugees and,refugee population,2.667,0,0
4,measures and,"instruments , and",3.000,0,0
...,...,...,...,...,...
993,unless specified otherwise,unless specifically stated otherwise,4.667,0,0
994,its efforts to combat,the area of combating,3.667,0,0
995,considered by the,give consideration to the,4.333,0,0
996,an enlarged security,an expanded security,4.000,0,0


In [41]:
def preprocess(raw_text,stopwords_remove=True):
    # keep only words
    letters_only_text = re.sub("[^a-zA-Z]", " ", str(raw_text))
    # convert to lower case and split 
    words = letters_only_text.lower().split()
    #print(words)
    # remove stopwords
    stopword_set = set(stopwords.words("english"))
    cleaned_words = list(set([w for w in words if w not in stopword_set]))
    return cleaned_words if stopwords_remove else words

def BLEU1score(s1, s2,stopwords_remove=True):

    hypothesis = [word for word in preprocess(s1,stopwords_remove)]
    reference = [word for word in preprocess(s2,stopwords_remove)]
    #print(hypothesis,",",reference)
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, weights=(1, 0, 0, 0))
    return BLEUscore


In [42]:
scores = []
for i in range(df.shape[0]):
    s1 = str(df['text_1'][i])
    s2 = str(df['text_2'][i])
    #print(s1,",",s2)
    dis = BLEU1score(s1,s2,stopwords_remove=False)
    df.loc[i,'bleu_allwords']=dis


In [43]:
scores = []
for i in range(df.shape[0]):
    s1 = str(df['text_1'][i])
    s2 = str(df['text_2'][i])
    #print(s1,",",s2)
    dis = BLEU1score(s1,s2,stopwords_remove=True)
    df.loc[i,'bleu_withoutstop']=dis

In [44]:
df.head(20)


,text_1,text_2,label,bleu_allwords,bleu_withoutstop
0,take measures in order to,take additional measures to,4.000,0.600000,0.666667
1,was seeking to,will strive to,3.000,0.333333,0.000000
2,canada aims to,canada works to,2.333,0.666667,0.500000
3,refugees and,refugee population,2.667,0.000000,0.000000
4,measures and,"instruments , and",3.000,0.500000,0.000000
5,provide both,", is provided",3.667,0.000000,0.000000
6,policy goals,political goals,4.333,0.500000,0.500000
7,of non-governmental organisations,for non-governmental organizations,4.000,0.500000,0.666667
8,of the service,of services are,3.333,0.333333,0.000000
9,resource availability .,of the resources available .,5.000,0.000000,0.000000


## Just some experiments with bleu

In [28]:
import nltk
hypothesis = ['This', 'is', 'cat'] 
reference = ['This', 'is', 'a', 'cat']
references = [reference] # list of references for 1 sentence.
list_of_references = [references] # list of references for all sentences in corpus.
list_of_hypotheses = [hypothesis] # list of hypotheses that corresponds to list of references.
print(nltk.translate.bleu_score.corpus_bleu(list_of_references, list_of_hypotheses))
print(nltk.translate.bleu_score.sentence_bleu(references, hypothesis))


8.987727354491445e-155
8.987727354491445e-155


In [33]:
>>> from nltk.translate import bleu

from nltk.translate.bleu_score import SmoothingFunction
hyp =  ['take', 'additional', 'measures', 'to']
ref = ['take', 'measures', 'in', 'order', 'to'] 
smoothie = SmoothingFunction().method4
bleu([ref], hyp, smoothing_function=smoothie)


1.6736959844303068

In [34]:
# 1-gram individual BLEU
from nltk.translate.bleu_score import sentence_bleu
reference = [['this', 'is', 'small', 'test']]
candidate = ['this', 'is', 'a', 'test']
score = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
print(score)

0.75


In [37]:
# n-gram individual BLEU
from nltk.translate.bleu_score import sentence_bleu
reference = [['take', 'measures', 'in', 'order', 'to']] 
candidate = ['take', 'additional', 'measures', 'to']
print('Individual 1-gram: %f' % sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
print('Individual 2-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 1, 0, 0)))
print('Individual 3-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 1, 0)))
print('Individual 4-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 0, 1)))

Individual 1-gram: 0.584101
Individual 2-gram: 0.000000
Individual 3-gram: 0.000000
Individual 4-gram: 0.000000


## Glue STS

In [62]:
import codecs

doc = codecs.open("e:\\WORK\\ML\\data\\glue\\stsbenchmark\\sts-train.csv",'rU','UTF-8') #open for reading with "universal" type set

df = pd.read_csv(doc, sep='\t', error_bad_lines=False, 
                 names=['cat1', 'cat2', 'year', 'idx', 'score','text_1','text_2','aux1','aux2'],
                usecols=['text_1','text_2','score'])

In [63]:
df

,score,text_1,text_2
0,5.00,A plane is taking off.,An air plane is taking off.
1,3.80,A man is playing a large flute.,A man is playing a flute.
2,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,2.60,Three men are playing chess.,Two men are playing chess.
4,4.25,A man is playing the cello.,A man seated is playing the cello.
...,...,...,...
5706,0.00,Severe Gales As Storm Clodagh Hits Britain,Merkel pledges NATO solidarity with Latvia
5707,0.00,Dozens of Egyptians hostages taken by Libyan t...,Egyptian boat crash death toll rises as more b...
5708,0.00,President heading to Bahrain,President Xi: China to continue help to fight ...
5709,0.00,"China, India vow to further bilateral ties",China Scrambles to Reassure Jittery Stock Traders


In [64]:
df['bleu_allwords'] = 0
df['bleu_withoutstop'] = 0

In [65]:
scores = []
for i in range(df.shape[0]):
    s1 = str(df['text_1'][i])
    s2 = str(df['text_2'][i])
    #print(s1,",",s2)
    dis = BLEU1score(s1,s2,stopwords_remove=False)
    df.loc[i,'bleu_allwords']=dis


e:\WORK\ML\YDATA\NLP\assignment_research\nlprs\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
e:\WORK\ML\YDATA\NLP\assignment_research\nlprs\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
e:\WORK\ML\YDATA\NLP\assignment_research\nlprs\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Con

In [66]:
scores = []
for i in range(df.shape[0]):
    s1 = str(df['text_1'][i])
    s2 = str(df['text_2'][i])
    #print(s1,",",s2)
    dis = BLEU1score(s1,s2,stopwords_remove=True)
    df.loc[i,'bleu_withoutstop']=dis

In [67]:
df.head(20)

,score,text_1,text_2,bleu_allwords,bleu_withoutstop
0,5.000,A plane is taking off.,An air plane is taking off.,0.654985,0.606531
1,3.800,A man is playing a large flute.,A man is playing a flute.,0.857143,0.750000
2,3.800,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...,0.695986,0.654985
3,2.600,Three men are playing chess.,Two men are playing chess.,0.800000,0.750000
4,4.250,A man is playing the cello.,A man seated is playing the cello.,0.846482,0.716531
5,4.250,Some men are fighting.,Two men are fighting.,0.750000,0.606531
6,0.500,A man is smoking.,A man is skating.,0.750000,0.500000
7,1.600,The man is playing the piano.,The man is playing the guitar.,0.833333,0.666667
8,2.200,A man is playing on a guitar and singing.,A woman is playing an acoustic guitar and sing...,0.666667,0.584101
9,5.000,A person is throwing a cat on to the ceiling.,A person throws a cat on the ceiling.,0.700000,0.750000
